# Module 2 Challenge - Main Notebook for PyBank Challenge

Due Date: Thursday 7 September 2023

## Summary
This script analyses financial records contained in a csv file (configured with INPUT_PATH and INPUT_DATA_FILENAME settings). It then 
genenrates a report (which is output to the terminal and to the a text file (configured with OUTPUT_PATH and OUTPUT_DATA_FILENAME settings).

## Assumptions
1. There will always be 2 columns present in the order provided in the sample data
2. A header column is present (could down the track add more complex validation to automatically detect header presence)
The csv file is expected to contain:
* A Header Row with two columns "Date" and "Profit/Losses"
* "Date" column in the format of MMM-YYYY (for example Jan-2011)
* "Profit/Losses" column numeric type (rounded to whole numbers) which may be positive or negative.
* Records are sorted in ascending order by Date

For example:
```
Date,Profit/Losses
Jan-2010,867884
Feb-2010,984655
Mar-2010,322013
Apr-2010,-69417
```

## Pseudo Code
1. Initialise
    1. Intialise variables (eg counters etc)
1. Process Data
    1. Checks input file exists. Abort on error    
    1. Load the input file. ~~(COULD: on error abort with appropriate error message)~~
    1. ~~COULD: Check csv file has at least 2 columns. if < 2 Abort with appropriate error message.~~
    1. Check row 1 headings are present
    1. ~~COULD: determine which is the date column and which is the numeric value automatically by the header column name if present.~~   
    1. For each row of the CSV file
        1. Perform the financial calculations (as per Detail section below)
            1. Skip the header row if present (ie read next record) 
            1. Increment the Record Count
            1. Add the Profit/Loss Amount from the current record to the Total Profit/Loss Amount
            1. If this is not the first record (ie Record Count is greater than 1)
                1. Calculate the Month to Month Change Amount by subtracting the current record's  P/L Amount from the Previous Month's Amount
                1. Add the Month to Month Change Amount to the Total Month to Month Change Amount
                1. If the Month to Month Change Amount is greater than the Greatest Increase Amount previously recorded then
                    1. set the Greatest Increase Amount to the current record's Month to Month Change Amount
                    1. set the Greatest Increase Date to  the current records's Date 
                1. If the Month to Month Change Amount is less than the Greatest Decrease Amount previously recorded then
                    1. set the Greatest Decrease Amount to the current record's Month to Month Change Amount
                    1. set the Greatest Decrease Date to  the current records's Date
            1. Read next record
    1. If the Record CountRecord Count is greater than 0 then
        1. Calculate the Average Monthly Change by dividing the Total Month to Month Change Amount by the number of Record Count
        1. Output the results to terminal
        1. Output the results to file
    1. Else
        Output a message to screen saying no records found
1. Finalise
    1. Tidy up (if required)

### Detail
The following calculations are performed from the data contained in the input file "budget_data.csv":
1. The total number of months included in the dataset (ie the number of records)
2. The net total amount of Profit/Losses over the entire period: (sum of all the records)
3. The average of the changes in Profit/Losses over the entire period: ( e.g. round ( average( y2 - y1, y3 - y2, yn - y(n-1) ) )
4. The greatest increase in profits (date and amount) over the entire period ( maximum of the calculated average changes )
5. The greatest decrease in losses (date and amount) over the entire period ( minimum of the calculated average changes )


Output
1. The results are displayed on the teminal and exported to a text file.
1. ~~COULD:~~   
    1. ~~warn the user if they want to overwrite the output file if it already exists; OR~~
    1. ~~name the output file  (financial_report_MIN-DATE_MAX-DATE.csv) OR~~
    1. ~~use a function to generate a unique filename.~~
 
The output report  similar to the following:

```
Financial Analysis
----------------------------
Total Months: 99    
Total: $-9999999999    
Average  Change: $-9999999999.99    
Greatest Increase in Profits: MMM-YYYY ($-9999999999)    
Greatest Decrease in Profits: MMM-YYYY ($-9999999999)    
```

# References / Acknowledgements
1. Python Language Reference: https://docs.python.org/3/library/functions.html
2. Class notes and concepts from: USYD-VIRT-FIN-PT-07-2023-U-LOLC\01-Lessons\02-Python\3\Activities\09-Ins_CSV_Reader\Solved\csv_reader.py

In [4]:
# Module 2 Challenge - Main Notebook for PyBank Challenge
"""Summary: This script analyses financial records contained in a csv file then outputs the financial analysis results to a text file and to the terminal.
"""     
# Import the required libraries (ie pathlib and csv)
from pathlib import Path
import os
import csv

In [5]:
# Initialise settings. Note: a convention of UPPERCASE has been adopted for constants

# DEBUG: Shows debug messages when set to True
# --------------------------------------------
DEBUG = False

# INPUT_PATH: folder name where the input file resides
# ----------------------------------------------------
INPUT_PATH = "./Resources/"

# INPUT_DATA_FILENAME: filename containing the input data in csv format for the calculations
# ------------------------------------------------------------------------------------------
INPUT_DATA_FILENAME = "budget_data.csv"

# CSV_FILE_DELIMITER: delimiter used in the csv file to separate columns
# ----------------------------------------------------------------------
CSV_FILE_DELIMITER = ','

# CSV_HAS_HEADER: (boolean) True if we are expecting a header in the csv file. False if not
# -----------------------------------------------------------------------------------------
CSV_HAS_HEADER = True

# OUTPUT_PATH: folder name where the output report will be saved
# --------------------------------------------------------------
OUTPUT_PATH = "./"

# OUTPUT_REPORT_FILENAME: filename the output report will be saved
# --------------------------------------------------------
OUTPUT_REPORT_FILENAME = "budget_report.txt"

In [6]:
def process_csv():
    """Summary: This script analyses financial records contained in a csv file then outputs the financial analysis results to a text file and to the terminal.
    Function process_csv loads the input csv file and performs financial calculations using the data from the csv file
    Parameters:
        none
    Returns:
        True if completed successfully
        False if errors were encountered
    """     

# =========================================
# Initialise local variables
# =========================================
    # For financial report output
    total_profit_loss_amount = 0             # Sum of all the Profit/Loss of each month

    greatest_increase_amount = 0             # Greatest increase amount from one month to the next
    greatest_increase_date = ""              # Date of greatest increase
    greatest_decrease_amount = 0             # Greatest decrease amount from one month to the next
    greatest_decrease_date = ""              # Date of decrease increase

    
    # Working variables for calculations
    row_number = 0                           # Row count for the number of lines in the csv file
    record_number = 0                        # Record number (excluding the header row if expected)
    previous_month_amount = 0                # Amount from the previous month. 
    month_to_month_change_amount = 0         # Amount of change from the previous month to the current month
    total_month_to_month_change_amount = 0   # Running total of month to month changes used to calculate the average change

    # File handling variables
    input_filename = ""                      # Operating system independent absolute path and filename to the input file
    output_filename = ""                     # Operating system independent absolute path and filename to the out file

    csv_date = ""                            # Temporary buffer to store the Date value of the current record 
    csv_amt = 0                              # Temporary buffer to store the Profit/Loss amount of the current record  

    # For reporting results
    output_text = ""                         # Buffer where the results are formatted into and used for both output to terminal and to the output report file

# =========================================
# Main body of code
# =========================================
 
    if DEBUG:
        print("Settings...")
        print(f"Has Header Row: '{CSV_HAS_HEADER}'")
        print(f"Columns separated by '{CSV_FILE_DELIMITER}'")

    # Obtain a path to the input and output files which is compatible with the user's operating system
    input_filename = os.path.abspath(INPUT_PATH + INPUT_DATA_FILENAME)
    output_filename = os.path.abspath(OUTPUT_PATH + OUTPUT_REPORT_FILENAME)

    if DEBUG:
        print(f"Input data filename: {input_filename}")
        print(f"Output report filename: {output_filename}")
    
    # Check if the input file exists, if not show an error and abort
    if (not os.path.exists(input_filename)):
        print(f'Error: Input file "{input_filename}" does not exist')
        return False

    # Get a file object for the input file
    with open( input_filename, 'r') as input_file_obj:
        if DEBUG:
            print(f"Input File Object Type: {type(input_file_obj)}")

        # Get a csv object from the input file object
        input_csv_obj = csv.reader(input_file_obj, delimiter=CSV_FILE_DELIMITER)

        if DEBUG:
            print(f"Input CSV Object Type: {type(input_csv_obj)}")
        
        # Read each row of the csv file
        for csv_row_data in input_csv_obj:
            row_number += 1      # increment row counter after we read a row in from the csv file
            if DEBUG:
                print(f"Row #{row_number}:{csv_row_data}")  # output the row number and row content
            
            # If a header row is expected and we are at the first row then skip over it and go to the next record
            if (CSV_HAS_HEADER and row_number == 1):
                if DEBUG:
                    print("Skipping header record")
                continue        # skip to next record

            record_number += 1   #  increment record count for all rows (except the header)
            
            csv_date = csv_row_data[0]         # place the date field from the csv record into a local variable for code legibility
            csv_amt = int(csv_row_data[1])     # place the profit/loss amount field from the csv record into a local variable for code legibility and convert string to integer
            
            total_profit_loss_amount += csv_amt    # add the amount from the current record to the to the Total Profit/Loss Amount
            if (record_number > 1):
                month_to_month_change_amount = csv_amt - previous_month_amount       # Calculate the Month to Month Change Amount by subtracting the current record's  P/L Amount from the Previous Month's Amount
                total_month_to_month_change_amount += month_to_month_change_amount   # Add the Month to Month Change Amount to the Total Month to Month Change Amount
                
                # work out if this month's increase is the highest, if so keep the amount and date
                if (month_to_month_change_amount > greatest_increase_amount):     # If the Month to Month Change Amount is greater than the Greatest Increase Amount previously recorded then
                    greatest_increase_amount = month_to_month_change_amount       #   set the Greatest Increase Amount to the current record's Month to Month Change Amount
                    greatest_increase_date = csv_date                             #   set the Greatest Increase Date to the current records's Date 

                    
                # work out if this month's decrease is the highest, if so keep the amount and date
                if (month_to_month_change_amount < greatest_decrease_amount):     # If the Month to Month Change Amount is less than the Greatest Decrease Amount previously recorded then
                    greatest_decrease_amount = month_to_month_change_amount       #   set the Greatest Decrease Amount to the current record's Month to Month Change Amount
                    greatest_decrease_date = csv_date                             #   set the Greatest Decrease Date to the current records's Date 
                
            previous_month_amount = csv_amt    # Set the "previous month's amount" with the current record's amount, which will be used when processing the next record
        # ---- end for
    # ---- end with

    # Output the results...
    if (record_number > 0): # Only show financial analysis report if there was at least one record 
        # format the same output_text for output to terminal and to a file 
        output_text = f"Financial Analysis\n----------------------------\nTotal Months: {record_number}\nTotal: ${total_profit_loss_amount}\n"
        if (record_number > 1):  # Only show monthly comparative results if we processed more than 1 record (as there is nothing to compare with otherwise)
            output_text += f"Average  Change: ${round( total_month_to_month_change_amount / (record_number - 1),2)}\n"
            output_text += f"Greatest Increase in Profits: {greatest_increase_date} (${greatest_increase_amount})\n"
            output_text += f"Greatest Decrease in Profits: {greatest_decrease_date} (${greatest_decrease_amount})\n"        
    else:
        output_text = "Warning: no financial records found" # Informative message if no records were found


    if DEBUG:
        print("\n\n") # leave some space between debug messages and the actual output of the financial report

    # Output the results to the terminal
    print(output_text)
                
    # Save the results as a text file (whether we found any records or not)
    with open(output_filename, 'w') as report_file_handle:
        report_file_handle.write(output_text)
    
    # Tell the user where they can find the file with the report
    print(f'A copy of the report has been saved to: "{output_filename}" ')
        
    return True   # Return True to signal successful completion

In [ ]:
# Run the process_csv function
result = process_csv()